<a href="https://colab.research.google.com/github/jongchan1514/keras/blob/main/keras_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras

# 케라스에서 MNIST 데이터셋 준비하기

In [12]:
from keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images.shape
len(train_images)

60000

# IMDB 데이터셋

In [2]:
#인터넷 영화 데이터베이스
from keras.datasets import imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

#단어와 정수 인덱스를 매핑한 딕셔너리
word_index = imdb.get_word_index()

#정수 인덱스와 단어를 매핑하도록 뒤집기
reverse_word_index = dict(
    [(value, key) for (key, value) in word_index.items()])

#리뷰 디코딩
decoded_review = ' '.join(
    [reverse_word_index.get(i - 3, '?') for i in train_data[0]])

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

In [3]:
#신경망에 주입 할 데이터준비 one-hot vector
import numpy as np

def vectorrize_sequences(sequences, dimension=10000):
  #크기가 len(sequences), dimension 이고 모든 원소가 0인 행렬생성
  results = np.zeros((len(sequences), dimension))
  for i, sequences in enumerate(sequences):
    #results[i]에서 특정인덱스의 위치를 1로만들기
    results[i, sequences] = 1.
  return results

#훈련데이터를 벡터로 변환
x_train = vectorrize_sequences(train_data)

#테스트데이터를 벡터로 변환
x_test = vectorrize_sequences(test_data)

In [38]:
#샘플
x_train[0]

array([0., 1., 1., ..., 0., 0., 0.])

In [4]:
#레이블을 벡터로 변환
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

In [5]:
#모델 정의하기
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [48]:
#모델 컴파일
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [41]:
#옵티마이저 설정
from keras import optimizers

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])


#손실과 측정을 함수객체로 지정
from keras import losses
from keras import metrics
model.compile(optimizer=optimizers.RMSprop(lr=0.001),
              loss=losses.binary_crossentropy,
              metrics=[metrics.binary_accuracy])

In [6]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

In [7]:
#훈련검증 세트준비
x_val = x_train[:10000]
partial_x_train = x_train[10000:]

y_val = y_train[:10000]
partial_y_train = y_train[10000:]

In [8]:
#모델 훈련하기
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val))

Epoch 1/20
30/30 [==============================] - 2s 56ms/step - loss: 0.5892 - acc: 0.7107 - val_loss: 0.4010 - val_acc: 0.8497
Epoch 2/20
30/30 [==============================] - 1s 36ms/step - loss: 0.3222 - acc: 0.9015 - val_loss: 0.3066 - val_acc: 0.8862
Epoch 3/20
30/30 [==============================] - 1s 36ms/step - loss: 0.2272 - acc: 0.9279 - val_loss: 0.2776 - val_acc: 0.8910
Epoch 4/20
30/30 [==============================] - 1s 36ms/step - loss: 0.1720 - acc: 0.9456 - val_loss: 0.2740 - val_acc: 0.8904
Epoch 5/20
30/30 [==============================] - 1s 35ms/step - loss: 0.1354 - acc: 0.9596 - val_loss: 0.2922 - val_acc: 0.8850
Epoch 6/20
30/30 [==============================] - 1s 36ms/step - loss: 0.1120 - acc: 0.9655 - val_loss: 0.2954 - val_acc: 0.8856
Epoch 7/20
30/30 [==============================] - 1s 36ms/step - loss: 0.0887 - acc: 0.9783 - val_loss: 0.3157 - val_acc: 0.8837
Epoch 8/20
30/30 [==============================] - 1s 37ms/step - loss: 0.0727 - a

In [9]:
#history객체 확인
history_dict = history.history
history_dict.keys()

dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])

In [ ]:
#훈련과 검증 손실 그리기
import matplotlib.pyplot as plt

history_dict = history.history
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(loss) + 1)

#bo 는 파란색 점
plt.plot(epochs, loss, 'bo', label='Training loss')